In [ ]:
# FOR CLEANING SCRIPT:

# ensure each asset does not appear before first date nor after last date
# cmc_ids = np.unique(panel_df.cmc_id.values)
# for cmc_id in cmc_ids:
#     print(cmc_id)
#     first_date = cw_df[cw_df.cmc_id==cmc_id].first_date_cmc.values[0]
#     last_date  = cw_df[cw_df.cmc_id==cmc_id].last_date_cmc.values[0]
#     assert(0==panel_df[(panel_df.cmc_id==cmc_id)&(panel_df.date<first_date)].shape[0])
#     assert(0==panel_df[(panel_df.cmc_id==cmc_id)&(panel_df.date>last_date)].shape[0])

# ensure each asset has consecutive data, interpolate where needed with forward fill

# group the data by cmc_id to loop over
grouped = panel_df.groupby('cmc_id')

# interate through each cmc_id
dfs = []
for name, group in grouped:
    # find the first and last dates for the current id
    first_date = group['date'].min()
    last_date  = group['date'].max()

    # create a new dataframe with all the possible combinations of cmc_id and date
    dates = pd.date_range(first_date, last_date)
    index = pd.MultiIndex.from_product([[name], dates], names=['cmc_id', 'date'])
    full_df = pd.DataFrame(index=index).reset_index()

    # merge the full dataframe with the original dataframe to fill in missing values with NaNs
    merged_df = pd.merge(full_df, group, on=['cmc_id', 'date'], how='left')

    # interpolate the missing values using forward fill for up to 7 consecutive observations
    interpolated_df = merged_df.fillna(method='ffill', limit=21)

    # Check if there are any missing values in the remaining columns for the current id and date range
    if interpolated_df.isnull().values.any():
        print(f"ID {name} has missing values in the given date range, precisely: {int(interpolated_df.isnull().sum().sum()/3)}.")
        break

    # combine    
    dfs.append(interpolated_df)

# Combine all the dataframes and drop the 'cmc_id' index level
result_df = pd.concat(dfs).reset_index()


